# Parsing Debug Notebook

This somewhat messy notebook makes it easier to debug the parser, because we can just rerun the cells needed to set up the debugging process with various parameters.

In [1]:
import json
import math
import re
from pathlib import Path

import boto3
import pandas as pd

from parse_990_textract.bucket import open_df
from parse_990_textract.filing import create_roadmap, extract_from_roadmap
from parse_990_textract.models import BoundingBox, TableExtractor
from parse_990_textract.parse import create_extractors, find_item, find_pages
from parse_990_textract.postprocessing import clean_filing, clean_f_i, clean_f_ii, clean_f_iii, postprocess
from parse_990_textract.setup import load_extractor_df
from parse_990_textract.table import extract_table_data, find_table_pages, create_tablemap
from parse_990_textract.utils import get_coordinate, get_regex, cluster_words, columnize, cluster_x

In [2]:
bucket = boto3.resource("s3").Bucket("s3-ocr-990s-demo")

Setting `VALIDATE_TOP` to `True` will parse all 25 validation PDFs and compare non-Schedule F output to the validation data. If set to `False`, we run the test code for Schedule F instead.

In [3]:
VALIDATE_TOP = True

In [4]:
if VALIDATE_TOP:
    validation_data = pd.read_csv("validation_data.csv", index_col="job_id").fillna("")
    validation_data.head()

In [5]:
extractor_df = load_extractor_df("990_extractors.csv")
roadmap_df = load_extractor_df("990_roadmap.csv")
schedule_f_tablemap_df = load_extractor_df("schedule_f_table_roadmap.csv")
schedule_f_table_extractor_df = pd.read_csv("schedule_f_table_extractors.csv")
schedule_f_row_extractor_df = pd.read_csv("schedule_f_row_extractors.csv")

In [6]:
PART_I_HEADER = r"\(a\)\s*Region|\(d\)\s*Activities|\(e\)\s*If activity|\(f\)Total expenditures"
PART_II_HEADER = r"\(b\)\s*IRS code|\(c\)\s*Region|\(d\)\s*Purpose|\(f\)\s*Manner|\(h\)\s*Description"
PART_III_HEADER = r"\(b\)\s*Region|\(e\)\s*Manner of cash|\(h\)\s*Method of va"
PART_I_TABLE_NAME = r"Activities per Region"
PART_II_TABLE_NAME = r"Grants to Organizations Outside the United States"
PART_III_TABLE_NAME = r"Grants to Individuals Outside the United States"

In [7]:
filing_rows = []
schedule_f_part_i_rows = []
schedule_f_part_ii_rows = []
schedule_f_part_iii_rows = []
if VALIDATE_TOP:
    values = validation_data.index.values
else:
    values = []

for i, job_id in enumerate(values):
    print(i)
    print(job_id)
    pdf_key = validation_data.at[job_id, "pdf_key"]
    print(pdf_key)
    
    data = open_df(bucket, job_id)
    lines = data.loc[data["BlockType"] == "LINE"]
    words = data.loc[data["BlockType"] == "WORD"]
    page_map = find_pages(lines)
    roadmap = create_roadmap(
        lines, roadmap_df, page_map
    )
    
    row = extract_from_roadmap(
        words, lines, roadmap, extractor_df, page_map
    )
    row = postprocess(row, job_id, pdf_key, clean_filing)
    filing_rows.append(row)
    
    pages = lines.groupby("Page")
    
    part_i_table = extract_table_data(
        pages, lines, words, PART_I_HEADER, PART_I_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_i_table = postprocess(part_i_table, job_id, pdf_key, clean_f_i)
    if part_i_table is not None:
        schedule_f_part_i_rows.append(
            part_i_table
        )
    part_ii_table = extract_table_data(
        pages, lines, words, PART_II_HEADER, PART_II_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_ii_table = postprocess(part_ii_table, job_id, pdf_key, clean_f_ii)
    if part_ii_table is not None:
        schedule_f_part_ii_rows.append(
            part_ii_table
        )
    part_iii_table = extract_table_data(
        pages, lines, words, PART_III_HEADER, PART_III_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_iii_table = postprocess(part_iii_table, job_id, pdf_key, clean_f_iii)
    if part_iii_table is not None:
        schedule_f_part_iii_rows.append(
            part_iii_table
        )

0
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4
EIN_760733035_YEAR_2009_FORMTYPE_990.pdf


No match for year_formation in L Year of Formation M State of legal domicile
No match for state_of_domicile in M State of legal domicile


1
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc
EIN_363235550_YEAR_2009_FORMTYPE_990.pdf


No match for total_unrelated_biz_revenue in 7a NONE
No match for net_unrelated_biz_taxable_revenue in 7b NONE
No match for benefits_paid_members_expenses in NONE
No match for professional_fundraising_fees_expenses in NONE
No match for grants_foreign_individuals_govt_orgs_total in NONE
No match for benefits_to_members_total in NONE
No match for compensation_disq_persons_total in NONE
No match for fees_for_mgmt_services_total in NONE
No match for fees_for_lobbying_services_total in NONE
No match for fees_for_fundraising_services_total in NONE
No match for fees_for_investment_mgmt_services_total in NONE
No match for royalties_total in NONE
No match for entertainment_travel_govt_officials_total in NONE
No match for interest_total in NONE
No match for payments_affiliates_total in NONE


2
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b
EIN_223134995_YEAR_2010_FORMTYPE_990.pdf
3
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82
EIN_264320885_YEAR_2009_FORMTYPE_990.pdf


...

No match for activities_per_region_totals_number_of_offices in O


19
aefc7b65c34db330d8d9f56a1226e116b63ee9be7dbdd4ae4c7bea5d87359f97
EIN_362428692_YEAR_2009_FORMTYPE_990.pdf
20
a5a3cbfcf844be8862bbb61ad46d4c795891ab1143e420db5ed99fc79eeb66c9
EIN_271377148_YEAR_2016_FORMTYPE_990.pdf
21
67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebdb32d3186b19d6a6fd5
EIN_42103580_YEAR_2010_FORMTYPE_990.pdf
22
a6529e504df346097da99104a353e977426e018cd5ac33b62cd2dd89c90763c5
EIN_311002913_YEAR_2008_FORMTYPE_990.pdf
23
d1925c2d74adaa3f150ded3ce67dfe7ae3a306f0db0289ad5755a28d801b2b0b
EIN_202408857_YEAR_2010_FORMTYPE_990.pdf


No match for activities_per_region_totals_number_of_offices in Act the


24
01d89ee5d14575c1321b2e4d67431d172ba76212b4a266bdaf474275029fd78b
EIN_521830327_YEAR_2009_FORMTYPE_990.pdf


No match for total_fundraising_expenses in b Total fundraising expenses (Part IX, column (D), line 25)
No match for revenue_less_expenses in <43,775.> End of Year


In [9]:
if VALIDATE_TOP:
    output_data = pd.concat(filing_rows).reset_index(drop=True).set_index("job_id")

In [10]:
VALIDATE_TOP and output_data.head()

field_name,name,address,city,state,zip,website,gross_receipts,year_formation,state_of_domicile,mission,...,total_functional_expense_fundraising,activities_per_region_totals_number_of_offices,activities_per_region_totals_number_of_employees,activities_per_region_totals_total_expenditure,total_number_recipient_foreign_orgs_listed_as_charities,total_number_other_recipient_foreign_orgs_entities,pdf_key,ein,year,filing_id
job_id,,,,,,,,,,,,,,,,,,,,,
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4,MAKE WAY PARTNERS INC,PO BOX 26367,BIRMINGHAM,OX,26367,www MAKEWAYPARTNERS ORG,1426217,,,EVANGELICAL MISSIOI WORK TO PREVENT AND COMBAT...,...,4503,,,,,,EIN_760733035_YEAR_2009_FORMTYPE_990.pdf,760733035,2009,760733035_2009
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc,THE FEDERALIST SOCIETY FOR LAW AND,"1015 18TH ST., N.W. 425",WASHINGTON,DC,20036,FED-SOC.ORG,11033302,1982,IL,THE ORGANIZATION'S MISSION IS TO PROMOTE INTEL...,...,597789,,,,,,EIN_363235550_YEAR_2009_FORMTYPE_990.pdf,363235550,2009,363235550_2009
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b,Ron Hutchcraft Ministries Inc,PO Box 400,Harnson,AR,20400,www hutchcraft com,3390,1991,NJ,To communicate Christ to the lost in their lan...,...,263765,,,,,,EIN_223134995_YEAR_2010_FORMTYPE_990.pdf,223134995,2010,223134995_2010
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82,FREEDOM 4 24,21430 TIMBERLAKE ROAD STE 101,LYNCHBURG,VA,24502,,61857,2009,VA,Freedom 424s mission is to provide a pathway t...,...,669,,,,,,EIN_264320885_YEAR_2009_FORMTYPE_990.pdf,264320885,2009,264320885_2009
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be,GEORGIA PUBLIC POLICY FOUNDTION,6100 LAKE FORREST DR,LAKE FORREST,GA,30328,www GPPF org,583209,1991,GA,To further goals of economic growth & individu...,...,14409,,,,,,EIN_581943161_YEAR_2009_FORMTYPE_990.pdf,581943161,2009,581943161_2009


In [11]:
def clean(x):
    x = str(x)
    x = re.sub(r"\.0\b", "", x)
    x = re.sub("\D", "", x)
    return x

In [12]:
def compare_output(to_validate, to_compare, col):
    return pd.DataFrame(
        {
            "extracted": to_validate.loc[col].loc[
                lambda series: series != to_compare.loc[col]
            ],
            "expected": to_compare.loc[col].loc[
                lambda series: series != to_validate.loc[col]
            ],
        }
    )

In [13]:
if VALIDATE_TOP:
    to_compare = validation_data.set_index("pdf_key").applymap(clean)
    to_validate = output_data[validation_data.columns].set_index("pdf_key").applymap(clean)

    for col in to_validate.index:
        validated = compare_output(to_validate, to_compare, col)
        if validated.any().any():
            print(col)
            print(f"{validated.shape[0]} mismatched items.")
            print(validated)
            print("-"*79)

EIN_113489123_YEAR_2008_FORMTYPE_990.pdf
2 mismatched items.
              extracted expected
total_revenue      1017  1017506
travel_total        137    13710
-------------------------------------------------------------------------------
EIN_582248383_YEAR_2009_FORMTYPE_990.pdf
1 mismatched items.
                                                   extracted expected
total_number_other_recipient_foreign_orgs_entities        10         
-------------------------------------------------------------------------------


In [14]:
VALIDATE_TOP and pd.concat(schedule_f_part_i_rows).head(50)

field,region,number_offices,number_employees,activities_conducted,specific_type_activity,total_expenditures,job_id,pdf_key,ein,year,filing_id
0,Europe,1,0,theological training,offers a Th M degree,6661,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf,231352689,2009,231352689_2009
1,Totals,1,0,,,6661,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf,231352689,2009,231352689_2009
0,(1),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
1,(2),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
2,(3),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
3,(4),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
4,(5),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
5,(6),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
6,(7),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
7,(8),,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010


In [15]:
VALIDATE_TOP and pd.concat(schedule_f_part_i_rows).tail(50)

field,region,number_offices,number_employees,activities_conducted,specific_type_activity,total_expenditures,job_id,pdf_key,ein,year,filing_id
25,Europe :luding Icelan & Greenland,,,vestments,,,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
26,Middle East and North Africa,1,2,Grantmaking,,698609,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
27,Middle East and North Africa,,,Progr services,Service centers,26658,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
28,Middle East and North Africa,,,Progr services,Academic support,31354,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
29,Middle East and North Africa,,,Programservices,Institutional support,7637,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
30,Middle East and North Africa,,,Progr Services,Instruction,674753,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
31,Middle East and North Africa,,,Progr Services,Research other academicactivity,325683,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
32,Middle East and North Africa,,,Progr Services,Student Services,80648,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
33,Middle East and North Africa,,,Investment,,,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
34,North merica,,,Fundraising,,33431,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010


In [16]:
VALIDATE_TOP and pd.concat(schedule_f_part_ii_rows).head(50)

field,org_name,irs_code,region,grant_purpose,amount_cash,manner_cash,amount_noncash,desc_noncash,method_valuation,job_id,pdf_key,ein,year,filing_id
0,(1),,,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
1,(2),,,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
2,(3),,,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
3,(4),,,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
4,(5),,,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
5,(6),,,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
6,(7),,,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
7,(8),,,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
8,(9),,,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
9,(10),,,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010


In [22]:
VALIDATE_TOP and pd.concat(schedule_f_part_iii_rows).head(50)

field,type_of_grant_assistance,region,number_recipients,amount_cash_grant,manner_cash_disbursement,amount_noncash_assistance,desc_noncash_assistance,method_valuation,job_id,pdf_key,ein,year,filing_id
0,,,,,,,Schedule F,,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf,231352689,2009,231352689_2009
0,(1),,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
1,(2),,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
2,(3),,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
3,(4),,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
4,(5),,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
5,(6),,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
6,(7),,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
7,(8),,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
8,(9),,,,,,,,39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010


In [23]:
if VALIDATE_TOP:
    assert False

AssertionError: 

In [ ]:
table_test_df = open_df(bucket, "f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2f4a49dd0a81b456649")

In [ ]:
test_lines = table_test_df.loc[
    table_test_df["BlockType"] == "LINE"
]
test_words = table_test_df.loc[
    table_test_df["BlockType"] == "WORD"
]
test_pages = test_lines.groupby("Page")

In [ ]:
HEADER = PART_II_HEADER
NAME = PART_II_TABLE_NAME

In [ ]:
table_pages = find_table_pages(
    test_pages["Text"].agg(lambda words: " ".join(words)),
    HEADER,
)

In [ ]:
table_pages

In [ ]:
TEST_INDEX = 0
TEST_PAGE = 26

In [ ]:
tablemaps = pd.DataFrame(
    {
        "page": table_pages,
        "tablemap": table_pages.map(
            lambda page: create_tablemap(
                test_lines, schedule_f_tablemap_df, page, NAME
            ).dropna()
        ),
    }
)

In [ ]:
tablemaps["tablemap"].iloc[TEST_INDEX]

In [ ]:
row_extractors = schedule_f_row_extractor_df.loc[
    schedule_f_row_extractor_df["table"] == NAME
]
table_data = schedule_f_table_extractor_df.loc[
    schedule_f_table_extractor_df["table"] == NAME
].iloc[0]

In [ ]:
rows = tablemaps.assign(
    extractor=tablemaps["tablemap"].map(
        lambda tablemap: TableExtractor(
            header_top_label=table_data["header_top"],
            top_label=table_data["table_top"],
            bottom_label=table_data["table_bottom"],
            tablemap=tablemap,
            fields=row_extractors["field"],
            field_labels=row_extractors["col_left"]
        )
    )
)

In [ ]:
test_extractor = rows["extractor"].iloc[TEST_INDEX]

In [ ]:
test_extractor.get_col_spans(test_words, TEST_PAGE)

In [ ]:
test_extractor.field_labels

In [ ]:
test_extractor.header_top_label

In [ ]:
extracted = test_extractor.extract_rows(test_words, TEST_PAGE)

In [ ]:
extracted.head(25)

In [ ]:
table_words = test_extractor.get_table_words(test_words, TEST_PAGE)

In [ ]:
word_clusters = cluster_words(table_words, table_words["Height"].min(), "Midpoint_Y")
[" ".join(word.sort_values(by="Left")["Text"].values) for word in word_clusters]

In [ ]:
def columnize(word_cluster, col_spans):
    return col_spans.map(
        lambda span: word_cluster.loc[
            (word_cluster["Right"].between(*span, inclusive="right"))
        ]
    )


def get_cluster_coords(cluster):
    cluster_coords = {
        "Left": cluster["Left"].min(),
        "Right": cluster["Right"].max(),
        "Height": cluster["Height"].max(),
        "Midpoint_X": cluster["Midpoint_X"].median(),
        "Midpoint_Y": cluster["Midpoint_Y"].median(),
        "Top": cluster["Top"].min(),
        "Bottom": cluster["Bottom"].min(),
    }
    cluster_coords["Width"] = cluster_coords["Right"] - cluster_coords["Left"]
    return cluster_coords


def combine_row(row):
    return pd.Series([
        line.map(
            lambda x: x.sort_values(
                by="Left"
            ).reset_index(drop=True)["Text"].fillna("")
        ).agg(
            lambda x: " ".join(x.values)
        ) + " "
        for line in row
    ]).sum().str.strip()

col_spans = test_extractor.get_col_spans(test_words, TEST_PAGE)

col_spans

Row break scenarios:
1. Previous cluster and current cluster both have entries in the same numeric column
2. Alignment is TOP and current cluster has non-empty columns that are empty in previous row
3. Alignment is BOTTOM and current cluster has empty columns that are non-empty in previous row

In [ ]:
y_tol = table_words["Height"].median()
columnized = columnize(word_clusters[0], col_spans)
columnized.index = test_extractor.fields
last_col_coords = pd.DataFrame.from_records(
    columnized.map(
        get_cluster_coords
    )
)
rows = []
if NAME == PART_I_TABLE_NAME:
    numeric_cols = (1,2,5)
elif NAME == PART_II_TABLE_NAME:
    numeric_cols = (4,6)
elif NAME == PART_III_TABLE_NAME:
    numeric_cols = (2,3,5)
current_row = [columnized]
top_ws = (
    last_col_coords["Top"].min()
    - test_extractor.get_table_top(test_words, TEST_PAGE)
)
print(f"Y tolerance: {y_tol}")
print(f"Top whitespace: {top_ws}")
if top_ws > y_tol * 4:
    alignment = "BOTTOM"
else:
    alignment = "UNKNOWN"
print(f"Alignment: {alignment}")
print("First cluster:")
print(" ".join(word_clusters[0].sort_values(by="Left")["Text"].values))
for count, cluster in enumerate(word_clusters[1:]):
    print("-"*50)
    print(f"Alignment: {alignment}")
    print("Cluster:", " ".join(cluster.sort_values(by="Left")["Text"].values))
    columnized = columnize(cluster, col_spans)
    columnized.index = test_extractor.fields
    col_coords = pd.DataFrame.from_records(columnized.map(get_cluster_coords))
    nonempty = col_coords.dropna().index.to_series()
    last_nonempty = last_col_coords.dropna().index.to_series()
    print("Nonempty\n", nonempty)
    print("Last nonempty\n", last_nonempty)
    # more_cols true if current row has non-empty cells that
    # are empty in the preceding row
    more_cols = (~nonempty.isin(last_nonempty)).any()
    print("More cols:", more_cols)
    # less_cols true if last row has non-empty cells that
    # are empty in current row
    less_cols = (~last_nonempty.isin(nonempty)).any()
    print("Less cols:", less_cols)
    # both_numeric true if both rows have entries in numeric cols
    both_numeric = (
        nonempty.isin(numeric_cols) 
        & last_nonempty.isin(numeric_cols)
    ).any()
    y_delta = (
        col_coords["Top"].min()
        - last_col_coords["Bottom"].max()
    )
    print("Y delta", y_delta)
    print("Both numeric:", both_numeric)
    if (
        both_numeric
        or (more_cols and (alignment== "TOP"))
        or (less_cols and (alignment == "BOTTOM"))
        or (y_delta > y_tol)
    ):
        combined_row = combine_row(current_row)
        print(combined_row)
        rows.append(combined_row)
        current_row = [columnized]
    elif less_cols and (alignment == "UNKNOWN"):
        alignment = "TOP"
        current_row.append(columnized)
    elif more_cols and (alignment == "UNKNOWN"):
        alignment = "BOTTOM"
        current_row.append(columnized)
    else:
        current_row.append(columnized)
    last_col_coords = col_coords            

In [ ]:
header_words = test_extractor.get_header_words(test_words, TEST_PAGE)

In [ ]:
header_words.sort_values(by="Left")[["Text", "Left", "Right", "Midpoint_Y"]].tail(50)